In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [14]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score,train_test_split,GridSearchCV,KFold

In [15]:
X_test_raw = pd.read_csv("X_test_new_2.csv", index_col=0).values
X_train_raw = pd.read_csv("X_train_new_2.csv", index_col=0).values
y_train_raw = pd.read_csv("y_train.csv", index_col=0).values

In [18]:
from sklearn.preprocessing import StandardScaler,RobustScaler,PowerTransformer
def standardize_data(X_train,X_test) :
    r_train = RobustScaler()
    r_test = RobustScaler()
    
    p_train = PowerTransformer()
    r_train.fit(X_train)
    r_test.fit(X_test)
    
    scaled_train_data = r_train.transform(X_train)
    scaled_test_data = r_train.transform(X_test)
    
    return scaled_train_data,scaled_test_data

In [21]:
X_train,X_test = standardize_data(X_train_raw,X_test_raw)

In [22]:
clf = XGBClassifier(max_depth = 11,
                    n_estimators = 100,#110
                    learning_rate = 0.07,#0.09,
                    subsample = 0.8,#0.8,
                    colsample_bytree = 0.9,#0.8,
                    min_child_weight = 2.0,
                    gamma = 0.9,#0.6,
                    # reg_alpha = reg_alpha,
                    # reg_lambda = reg_lambda,
                    # scale_pos_weight = 1,
                    # objective = 'multi:softmax',
                    # num_class = 4,
                    eval_metric = 'auc',
                    use_label_encoder=False
)

ret = cross_val_score(clf, X_train, y_train_raw, scoring='f1_micro', cv=5)
print(ret)

clf.fit(X_train, y_train_raw)
print(np.sort(clf.feature_importances_))
sf = np.argsort(clf.feature_importances_)[::-1][:36]
print(sf)

strKFold = StratifiedKFold(n_splits=5,shuffle=False)

ret = cross_val_score(clf, X_train, y_train_raw, scoring='f1_micro', cv=strKFold)
print(ret)
print(np.mean(ret))

clf.fit(X_train, y_train_raw)
pred = clf.predict(X_test)
#clf.fit(X_train_raw[:,sf], y_train_raw)
#pred = clf.predict(X_test_raw[:,sf])

submission = np.hstack([np.arange(0, len(pred)).reshape(-1,1), pred.reshape(-1,1)]) 
submission_pd = pd.DataFrame(submission, columns=['id','y'])
submission_pd.to_csv('submission.csv', index=None)

[0.80078125 0.82128906 0.82600196 0.81524927 0.81524927]
[0.         0.         0.         0.         0.         0.00101945
 0.00112462 0.00112964 0.00113048 0.00115756 0.00120441 0.00121844
 0.00123708 0.0012458  0.00125423 0.00125452 0.00125566 0.0012578
 0.00126913 0.0012692  0.00127017 0.00127034 0.00127598 0.0012801
 0.00128196 0.00128871 0.00128906 0.00129013 0.00129222 0.0012948
 0.00130747 0.00131837 0.00132142 0.00132397 0.00133008 0.00133671
 0.00134253 0.0013448  0.00134944 0.00135243 0.00135424 0.00135498
 0.00135687 0.00135744 0.0013606  0.00136766 0.00137457 0.00138504
 0.00139479 0.00139494 0.00140337 0.00140709 0.00140888 0.00140942
 0.00141347 0.00141468 0.00141567 0.00141575 0.00141618 0.00142571
 0.00142644 0.00143093 0.00143212 0.00143229 0.00143266 0.00144509
 0.00144698 0.0014479  0.00144803 0.00144999 0.00145313 0.00145379
 0.00145478 0.00145496 0.00145952 0.00145961 0.0014607  0.00146558
 0.00146598 0.00146661 0.00146879 0.00147121 0.00147474 0.00147562
 0.00147

In [12]:
from xgboost import XGBClassifier
from hyperopt import fmin, tpe, hp,space_eval,rand,Trials,partial,STATUS_OK
results = []

def GBM(argsDict):
    max_depth = argsDict["max_depth"] + 7
    n_estimators = argsDict['n_estimators'] * 5 + 90
    learning_rate = argsDict["learning_rate"] * 0.02 + 0.05
    subsample = argsDict["subsample"] * 0.1 + 0.5
    colsample_bytree = argsDict["colsample_bytree"] * 0.1 + 0.7
    min_child_weight = argsDict["min_child_weight"] * 0.5
    gamma = argsDict["gamma"] * 0.1 + 0.5
    # reg_alpha = argsDict["reg_alpha"] * 0.1
    # reg_lambda = argsDict["reg_lambda"] * 0.1
    print("max_depth:" + str(max_depth))
    print("n_estimators:" + str(n_estimators))
    print("learning_rate:" + str(learning_rate))
    print("subsample:" + str(subsample))
    print("colsample_bytree:" + str(colsample_bytree))
    print("min_child_weight:" + str(min_child_weight))
    print("gamma:" + str(gamma))
    # print("reg_alpha:" + str(reg_alpha))
    # print("reg_lambda:" + str(reg_lambda))

    global X_train_1, y_train_1#, X_test

    gbm = XGBClassifier(max_depth = max_depth,
                        n_estimators = n_estimators,#110
                        learning_rate = learning_rate,#0.09,
                        subsample = subsample,#0.8,
                        colsample_bytree = colsample_bytree,#0.8,
                        min_child_weight = min_child_weight,
                        gamma = gamma,#0.6,
                        # reg_alpha = reg_alpha,
                        # reg_lambda = reg_lambda,
                        # scale_pos_weight = 1,
                        # objective = 'multi:softmax',
                        # num_class = 4,
                        eval_metric = 'auc',
                        use_label_encoder=False
    )

    strKFold = StratifiedKFold(n_splits=5,shuffle=False)
    cv_results = cross_val_score(gbm,X_train_raw[:,sf],y_train_raw,cv=strKFold,scoring="f1_micro") #X_train_1, y_train_1 
    metric = cv_results.mean()
    results.append(metric)
    print(cv_results)
    print(str(metric) + '\n')
    return -metric

space = {
    "max_depth":hp.randint("max_depth",5), # + 7
    "n_estimators":hp.randint("n_estimators",5),  # * 5 + 75
    "learning_rate":hp.randint("learning_rate",6),  #[0,1,2,3,4,5] -> 0.05,0.06
    "subsample":hp.randint("subsample",5),#[0,1,2,3] -> [0.7,0.8,0.9]
    "colsample_bytree":hp.randint("colsample_bytree",3),#[0,1,2,3,4] -> [0.7,0.8,0.9]
    "min_child_weight":hp.randint("min_child_weight",10), #[0,1,2,3,4,5,6,7] -> +1
    "gamma":hp.randint("gamma", 5), # * 0.1 + 0.5
    # "reg_alpha":hp.randint("reg_alpha", 30), # * 0.1
    # "reg_lambda":hp.randint("reg_lambda", 30), # * 0.1
    }
algo = partial(tpe.suggest,n_startup_jobs=1)
best = fmin(GBM,space,algo=algo,max_evals=100)#max_evals表示想要训练的最大模型数量，越大越容易找到最优解

print(best)
print(GBM(best))

max_depth:10                                                                    
n_estimators:100                                                                
learning_rate:0.13                                                              
subsample:0.8                                                                   
colsample_bytree:0.7999999999999999                                             
min_child_weight:4.5                                                            
gamma:0.9                                                                       
[0.79882812 0.80078125 0.80938416 0.79863148 0.80840665]                        
0.803206332478006                                                               

max_depth:10                                                                    
n_estimators:100                                                                
learning_rate:0.13                                                              
subsample:0.8              

learning_rate:0.09                                                              
subsample:0.9                                                                   
colsample_bytree:0.8999999999999999                                             
min_child_weight:4.0                                                            
gamma:0.6                                                                       
[0.80078125 0.8046875  0.82111437 0.79569892 0.80351906]                        
0.8051602211632453                                                              

max_depth:9                                                                     
n_estimators:90                                                                 
learning_rate:0.05                                                              
subsample:0.5                                                                   
colsample_bytree:0.7                                                            
min_child_weight:0.0       

KeyboardInterrupt: 